In [1]:
from google.colab import files
files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


{'sample_submission.csv': b'id,target\n0,0\n2,0\n3,0\n9,0\n11,0\n12,0\n21,0\n22,0\n27,0\n29,0\n30,0\n35,0\n42,0\n43,0\n45,0\n46,0\n47,0\n51,0\n58,0\n60,0\n69,0\n70,0\n72,0\n75,0\n84,0\n87,0\n88,0\n90,0\n94,0\n99,0\n101,0\n103,0\n106,0\n108,0\n111,0\n115,0\n116,0\n122,0\n123,0\n124,0\n125,0\n127,0\n140,0\n142,0\n147,0\n148,0\n150,0\n152,0\n154,0\n155,0\n166,0\n167,0\n169,0\n177,0\n179,0\n181,0\n186,0\n188,0\n189,0\n192,0\n200,0\n202,0\n206,0\n207,0\n214,0\n217,0\n223,0\n224,0\n227,0\n228,0\n230,0\n233,0\n234,0\n236,0\n239,0\n250,0\n255,0\n257,0\n259,0\n275,0\n278,0\n282,0\n284,0\n286,0\n288,0\n292,0\n295,0\n300,0\n304,0\n305,0\n306,0\n308,0\n311,0\n317,0\n319,0\n323,0\n324,0\n325,0\n326,0\n333,0\n339,0\n342,0\n343,0\n350,0\n351,0\n357,0\n359,0\n362,0\n366,0\n367,0\n369,0\n373,0\n374,0\n376,0\n377,0\n378,0\n379,0\n382,0\n385,0\n387,0\n388,0\n391,0\n392,0\n395,0\n399,0\n400,0\n403,0\n405,0\n408,0\n411,0\n414,0\n416,0\n417,0\n422,0\n425,0\n428,0\n430,0\n431,0\n433,0\n434,0\n439,0\n441,0\n4

In [2]:
! pip install -q tensorflow_text

     |████████████████████████████████| 3.4MB 19.4MB/s 


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.models import Model

In [4]:
%%time
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4", 
                         trainable=True, name='BERT-encoder')
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
                              name='preprocessing')

CPU times: user 31.9 s, sys: 5.12 s, total: 37 s
Wall time: 41.4 s


In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [6]:
weights = encoder.get_weights()

In [7]:
weights[0].shape

(30522, 1024)

In [8]:
embeddings = tf.keras.layers.Embedding(30522, 1024, name='embeddings')
embeddings.build((None, None))
embeddings.set_weights([weights[0]])
embeddings.trainable = False;

In [10]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
encoder_inputs = preprocessor(text_input)
ids, mask = encoder_inputs['input_word_ids'], encoder_inputs['input_mask']

x = embeddings(ids)
x = tf.keras.layers.Dropout(0.1, name='embedding_dropout')(x)
x = tf.keras.layers.Bidirectional(
        tf.keras.layers.RNN(
            tf.keras.layers.StackedRNNCells([
            tf.keras.layers.LSTMCell(units=256),
            tf.keras.layers.LSTMCell(units=256),
        ])), name='bi-LSTM')(x, mask=mask)
x = tf.keras.layers.Dense(units=256, activation=tf.keras.activations.swish)(x)
x = tf.keras.layers.Dropout(0.1, name='dense_dropout')(x)
x = tf.keras.layers.Dense(units=1,   activation='sigmoid')(x)
model = tf.keras.models.Model(inputs=text_input, outputs=x, name='fuck_twitter')
model.summary()

Model: "fuck_twitter"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 128, 1024)    31254528    preprocessing[0][2]              
__________________________________________________________________________________________________
embedding_dropout (Dropout)     (None, 128, 1024)    0           embeddings[0][0]                 
_______________________________________________________________________________________

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy')

In [12]:
import re

In [13]:
x = train.text.map(lambda s: re.sub(r'(https?|ftp):\/\/[^\s/$.?#].[^\s]*', ' ', s), None)

In [14]:
print(train.text.values[-1])
print(x.values[-1])

The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d
The Latest: More Homes Razed by Northern California Wildfire - ABC News  


In [15]:
train_history = model.fit(
    x=x, 
    y=train.target.values,
    validation_split=0.2,
    epochs=3
)

Epoch 1/3
191/191 [==============================] - 186s 939ms/step - loss: 0.6078 - accuracy: 0.6592 - val_loss: 0.4655 - val_accuracy: 0.7879
Epoch 2/3
191/191 [==============================] - 181s 946ms/step - loss: 0.4582 - accuracy: 0.8013 - val_loss: 0.4422 - val_accuracy: 0.8017
Epoch 3/3
191/191 [==============================] - 182s 951ms/step - loss: 0.4222 - accuracy: 0.8235 - val_loss: 0.4275 - val_accuracy: 0.7978


In [16]:
y = test.text.map(lambda s: re.sub(r'(https?|ftp):\/\/[^\s/$.?#].[^\s]*', ' ', s), None)

In [18]:
print(test.text.values[1243])
print(y.values[1243])

local family displaced by fire http://t.co/f1nu18CArd via @gofundme  Lets help that family out please
local family displaced by fire   via @gofundme  Lets help that family out please


In [20]:
submission['target'] = (model.predict(y, verbose=1) >= 0.5).astype(int)
submission.to_csv('res.csv', index=False)

102/102 [==============================] - 12s 116ms/step


In [21]:
submission.values[10:]

array([[   30,     0],
       [   35,     0],
       [   42,     0],
       ...,
       [10868,     1],
       [10874,     1],
       [10875,     1]])